In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
!pip3 install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

import datasets 
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
df = dataset['train'].to_pandas()
df.describe()


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,...,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,...,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,...,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,...,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,...,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,...,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


In [ ]:
df

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,37080,8590,2,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135552,22986,8303,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,True,True,False,False,False
135553,21008,6207,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135554,22986,7886,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,True,False,False,False,True,False


In [ ]:
text = df.text
labels = df.hate_speech_score

In [ ]:
import numpy as np
import pandas as pd 
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from PIL import Image
from sklearn.utils import resample
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from transformers import RobertaTokenizer
from transformers import TFRobertaModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Flatten
from sklearn.metrics import precision_recall_curve

import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve,auc
from keras import regularizers
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,f1_score,classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score
from itertools import cycle

lemma = WordNetLemmatizer()
stopword = set(STOPWORDS)
nltk.download('omw-1.4')
%matplotlib inline
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')


from nltk.corpus import brown
nltk.download("brown")
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
import datasets

# Load the dataset
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')
df = dataset['train'].to_pandas()

# Text cleaning
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: re.sub('\s+',' ',x))

# Stop words removal
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Stemming and Lemmatization
ps = PorterStemmer()
df['text'] = df['text'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
wnl = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: ' '.join([wnl.lemmatize(word) for word in x.split()]))



# Word Embedding (example using Word2Vec)
from gensim.models import Word2Vec

corpus = [text.split() for text in df_resampled['text']]
model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
df.hate_speech_score

0        -3.90
1        -6.52
2         0.36
3         0.26
4         1.54
          ... 
135551   -4.88
135552   -4.40
135553   -2.49
135554   -4.40
135555   -0.20
Name: hate_speech_score, Length: 135556, dtype: float64

In [ ]:
df.hate_speech_score = (round(df.hate_speech_score))
df['hate_speech_score'] = df['hate_speech_score'].apply(lambda x: 1 if x > 0 else 0)
df

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,37080,8590,2,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135552,22986,8303,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,True,True,False,False,False
135553,21008,6207,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135554,22986,7886,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,True,False,False,False,True,False


In [ ]:
data = df
data

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,39773,2790,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,47101,3379,3,4.0,4.0,4.0,4.0,4.0,4.0,0.0,...,False,False,False,False,True,False,False,False,True,False
3,43625,7365,3,2.0,3.0,2.0,1.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,12538,488,0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,37080,8590,2,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135552,22986,8303,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,False,True,True,False,False,False
135553,21008,6207,2,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
135554,22986,7886,2,2.0,0.0,0.0,0.0,2.0,0.0,0.0,...,False,False,False,False,True,False,False,False,True,False


In [ ]:
data = data.loc[:, ['text', 'hate_speech_score']]
data

,text,hate_speech_score
0,yes indeed she sort of reminds me of the elder...,0
1,the trans women reading this tweet right now i...,0
2,question these 4 broads who criticize america ...,0
3,it is about time for all illegals to go back t...,0
4,for starters bend over the one in pink and kic...,1
...,...,...
135551,___ __ breaking_news sayyed abdul...,0
135552,millions of yemenis participated in mass ralli...,0
135553,abeshinzo realdonaldtrump shinzoabe dictator ...,0
135554,millions of yemenis participated in mass ralli...,0


In [ ]:
import nltk
nltk.download('punkt')

import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Example dataframe
df = pd.DataFrame({'text': ['This is an example sentence with stopwords to be removed.',
                            'Another sentence with stopwords to be removed.']})

# Get set of English stopwords
stop_words = set(stopwords.words('english'))

# Define a function to remove stopwords from a sentence
def remove_stopwords(sentence):
    words = nltk.word_tokenize(sentence)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence

# Apply the function to all rows of the dataframe
data['text'] = data['text'].apply(remove_stopwords)

print(df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text
0  This is an example sentence with stopwords to ...
1     Another sentence with stopwords to be removed.


In [ ]:
data


,text,hate_speech_score
0,ye inde sort remind elder ladi play part movi ...,0
1,tran woman read tweet right beauti,0
2,question 4 broad critic america countri flee g...,0
3,time illeg go back countri origin keep freeway...,0
4,starter bend one pink kick pussi get tast beg ...,1
...,...,...
135551,___ __ breaking_new sayi abdulmalikth saudi re...,0
135552,million yemeni particip mass ralli 13squar var...,0
135553,abeshinzo realdonaldtrump shinzoab dictat goe ...,0
135554,million yemeni particip mass ralli 13squar var...,0


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df,out,max_len=96):
        
        self.df = df
        self.out = out
        self.max_len = max_len
        self.tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        
    def __getitem__(self,index):
        data = {}
        row = self.df.iloc[index]
        ids,masks,labels = self.get_input_data(row)

        #data['ids'] = torch.tensor(ids)
        data['ids'] = ids.clone().detach()

        data['masks'] = masks
        data['labels'] = self.out.iloc[index].values[0]
        data['labels'] = torch.tensor(data['labels'], dtype=torch.float32).clone().detach()

        #data['labels'] = torch.tensor(self.out.iloc[index].values[0],dtype=torch.float32)
        return data
    
    def __len__(self):
        return len(self.df)
    
    def get_input_data(self,row):
        ids = self.tokenizer.encode(row[0],add_special_tokens=True)
        pad_len = self.max_len - len(ids)
        if pad_len > 0 :
            ids += [0]*pad_len
        ids = torch.tensor(ids)    
        masks = torch.where(ids != 0 , torch.tensor(1),torch.tensor(0))
        return ids,masks,self.out.iloc[0].values
    
        

In [ ]:
data

In [ ]:
#df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',header=None)
train_set,val_set = train_test_split(data,test_size = 0.7)
train_set


,text,hate_speech_score
134377,happy pride month im gay url,0
68604,i wanted to show black women could be strong b...,0
118247,stop sucking that fatass dick you niggerloving...,1
118023,stupid asian bitch youre an idiot and you shou...,1
4830,own up to it however its not like im going to ...,0
...,...,...
119097,niggers and their liberal friends steal everyt...,1
13150,again another sad video china has been forcing...,0
14802,i was once in your exact position i hope thing...,0
8356,i feel not very much india can make up for the...,0


In [ ]:
train_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(train_set['text']),pd.DataFrame(train_set['hate_speech_score']),max_len=200),batch_size = 32, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(val_set['text']),pd.DataFrame(val_set['hate_speech_score']),max_len=200),batch_size = 32,num_workers = 2)

In [ ]:
# from sklearn.model_selection import train_test_split


# # Split the data into training and testing sets, with 80% for training and 20% for testing
# X_train, X_test, y_train, y_test = train_test_split(data.text, data.hate_speech_score, test_size=0.2, random_state=42)

# # train_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(X_train)),pd.DataFrame(y_train),max_len=200),batch_size = 32, shuffle = True, num_workers = 2)
# # val_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(X_test),pd.DataFrame(y_test),max_len=200),batch_size = 32,num_workers = 2)

# train_loader = torch.utils.data.DataLoader(Dataset(X_train, y_train, max_len=200), batch_size=32, shuffle=True, num_workers=2)
# val_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(X_test), pd.DataFrame(y_test), max_len=200), batch_size=32, num_workers=2)


In [ ]:
data

,text,hate_speech_score
0,yes indeed she sort of reminds me of the elder...,0
1,the trans women reading this tweet right now i...,0
2,question these 4 broads who criticize america ...,0
3,it is about time for all illegals to go back t...,0
4,for starters bend over the one in pink and kic...,1
...,...,...
135551,___ __ breaking_news sayyed abdul...,0
135552,millions of yemenis participated in mass ralli...,0
135553,abeshinzo realdonaldtrump shinzoabe dictator ...,0
135554,millions of yemenis participated in mass ralli...,0


In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc.item()

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        config = transformers.DistilBertConfig.from_pretrained('distilbert-base-uncased')
        self.distilBert = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased',config=config)
        self.fc0 = nn.Linear(768,512)
        self.d0 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512,256)
        self.d1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256,1)
        nn.init.normal_(self.fc0.weight,std= 0.1)
        nn.init.normal_(self.fc0.bias ,0.)
        nn.init.normal_(self.fc1.weight,std =0.1)
        nn.init.normal_(self.fc1.bias, 0.)
        nn.init.normal_(self.fc2.weight,std=0.1)
        nn.init.normal_(self.fc2.bias , 0.)

        
    def forward(self,input_ids,attention_mask):
        hid= self.distilBert(input_ids,attention_mask = attention_mask)
        hid= hid[0][:,0]
        x = self.fc0(hid)
        x = F.relu(x)
        x = self.d0(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.d1(x)
        return self.fc2(x)


In [ ]:
criterion = nn.BCEWithLogitsLoss(reduction='mean').to('cuda')
model = Model().to('cuda')
for params in model.distilBert.parameters():
    params.require_grad = False
    params._trainable = False
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

for epoch in range(3):
        
        epoch_loss = 0
        val_loss = 0
        correct = 0
        accuracy = 0

        for data in train_loader:
            ids = data['ids'].cuda()
            masks = data['masks'].cuda()
            labels = data['labels'].cuda()
            labels = labels.unsqueeze(1)
            model.train()
            optimizer.zero_grad()
            outputs = model(ids,masks)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
           
            epoch_loss += loss.item()
            
        print(f'Train Epoch {epoch} : Loss {epoch_loss/len(train_loader)}')
        print("Train Accuracy : ",binary_acc(outputs,labels))
        model.eval()
        correct = 0
        for data in val_loader:
            ids = data['ids'].cuda()
            masks = data['masks'].cuda()
            labels = data['labels'].cuda()
            outputs = model(ids,masks)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
        print(f'Val Epoch {epoch} : Loss {val_loss/len(val_loader)}')
        print("Val Accuracy : ",binary_acc(outputs,labels))
        


Train Epoch 0 : Loss 0.39911757835445677
Train Accuracy :  81.0
Val Epoch 0 : Loss 0.27041523203138196
Val Accuracy :  90.0
Train Epoch 1 : Loss 0.2473565361379733
Train Accuracy :  92.0
Val Epoch 1 : Loss 0.2226721574122178
Val Accuracy :  100.0
Train Epoch 2 : Loss 0.16267774687320827
Train Accuracy :  85.0
Val Epoch 2 : Loss 0.21567141467206738
Val Accuracy :  90.0


In [ ]:
def test(sentence):
    tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    ids = tokenizer.encode(sentence,add_special_tokens=True)
    padded = ids + [0]*(80 - len(ids))
    padded = torch.tensor(padded,dtype=torch.int64).unsqueeze(0)
    masks = torch.where(padded != 0 , torch.tensor(1), torch.tensor(0)).cuda()
    padded = padded.cuda()
    model.eval()
    output = model(padded,masks)
    return torch.round(F.sigmoid(output)).item()
    